# Favorite Destination Flight SMS and MMS Application

### Used Google API, Wolfram API and Twilio API

Google API - For Flight Search

Wolfram API - For Plotting on Map

Twilio API - For sending SMS and MMS

Mashape API - Phone Number Lookup API

##### Turn On your mobile internet data to receive MMS messages.

In [4]:
import pandas as pd
import requests
import json


# This function takes input for 2 variables. Airport Code and Phone number.
# Airport code needs to be exactly 3 letters
# Phone number needs to be exactly 10 digits
# When the input is valid, the program runs to the next part
# When the inputs are invalid, it runs again until user enters correct inputs
# input function will return Phone Number and Airport Code for further processing

def input_func():
    while True:
        fav_airport_code = input("Enter your Favorite City Airport Code: ")
        phone = input("Enter your Phone number to receive a flight plan MMS message: ")
        if(fav_airport_code=="quit" or phone=="quit"):
            print("Thank you for using the Flight SMS application")
            break
        elif(len(fav_airport_code)!=3 or fav_airport_code.isalpha()!=True or len(phone)!=10 or phone.isalpha()==True):
            print("Inputs error. Please put valid airport code and phone number")
        else:
            return (fav_airport_code, phone)
    return False
# This loop will take in user input information and run thru the API's
# Google Flights API will search flight between SYR airport and destination airport

while True:
    inputdata = input_func()
    if(inputdata==False):
        break
    else: # If all the inputs is correct, then API calls happen
        fav_airport_code = inputdata[0]
        phone = inputdata[1]
        GoogleAPI = "<API KEY>"
        
        url = "https://www.googleapis.com/qpxExpress/v1/trips/search?key="+GoogleAPI
        headers = {'content-type' : 'application/json'}

# GoogleAPI takes the data in payload variable for the post request
        payload = {
          "request": {
            "slice": [
              {
                "origin": "SYR",
                "destination": fav_airport_code,
                "date": "2017-06-05"
              }
            ],
            "passengers": {
              "adultCount": 1,
              "infantInLapCount": 0,
              "infantInSeatCount": 0,
              "childCount": 0,
              "seniorCount": 0
            },
            "solutions": 1,
            "refundable": False
          }
        }
# This post request is from Google API. Returns the flight information in  json format
        response = requests.post(url,  data=json.dumps(payload),headers=headers)
        
        # Response is stored in the flights dictionary
        flights = response.json()
        
        # From flights dictionaty, extract the value from the 'name' key.
        flight_name=flights['trips']['data']['carrier'][0]['name']
        
        # From flights dictionary, extract the cost 
        flight_cost = flights['trips']['tripOption'][0]['saleTotal']
        
        # From flights dictionary, extract flight data and time
        flight_date,flight_time = str(pd.to_datetime(flights['trips']['tripOption'][0]['slice'][0]['segment'][0]['leg'][0]['departureTime'])).split()
        
        # Construct a friendly message with name, data, time, cost information.
        text_message = "Hop on a " + flight_name + " plane on " + flight_date + " at " + flight_time + " from Syracuse Hancock International airport for just " + flight_cost + " to visit your dream destination. Happy Journey. Sent from Twilio API"
        
        # WolframAPI URL takes in the Airport code as input and plots a flight path and returns a PNG format image that's used by Twilio API to send as MMS to users.
        wolfram_url = "https://www.wolframcloud.com/objects/6be13892-cf98-4609-8e6f-3f246848e9f2?x="+fav_airport_code
        
        # After user enters phone no, store it in a text file
        file_name = "phone_numbers.txt"
        new_string = phone
        opened_file = open(file_name, 'a')
        opened_file.write("%r\n" %new_string)
        opened_file.close()
        # Append new phone numbers in the file with a new line
        
        
        # Import Client package from twilio.rest library to use Twilio Programmable SMS API
        from twilio.rest import Client

        # Find these values at https://twilio.com/user/account
        account_sid = "<YOUR ACCOUNT SID>"
        auth_token = "<YOUR AUTH TOKEN>"
        client = Client(account_sid, auth_token)
        # This code sends message with a friendly text and a Image that is retrieved from the Wolfram API.
        message = client.api.account.messages.create(to="+1"+phone,
                                             from_="+1<YOUR TWILIO PHONE NUMBER>",
                                             body=text_message,
                                             media_url=[wolfram_url
                                                   ])
        print("MMS message sending . . . Thank you")

Enter your Favorite City Airport Code: CMB
Enter your Phone number to receive a flight plan MMS message: 3154501435
MMS message sending . . . Thank you
Enter your Favorite City Airport Code: quit
Enter your Phone number to receive a flight plan MMS message: quit
Thank you for using the Flight SMS application


### Plotting phone number locations on a folium map

In [33]:
# Folium package is used for plotting on the Open Street Maps 
import folium

# We use this file with all the numbers  
file_name = "phone_numbers.txt"
with open(file_name) as f:
    content = f.readlines()
content = [x.strip('\'') for x in content]


CENTER_US = (39.8333333,-98.585522)
map = folium.Map(location=CENTER_US, zoom_start=4)

for pn in content:
    try:
        # params takes the one phone no at a time and sends it to the API to get the location
        params = { 'phone' : pn[0:9] }
        headers={ "X-Mashape-Key": "<Your API KEY>",
        "Accept": "application/json" }
        response = requests.get("https://cosmin-us-phone-number-lookup.p.mashape.com/get.php", params=params, headers=headers )
        phone_data = response.json()
        lat,lon=phone_data['latitude'],phone_data['longitude']
        pos = (lat,lon)
        # Marker adds the location to the map variable that's ready to be plotted
        marker = folium.Marker(location=pos, 
                    popup="%s, %s" % (phone_data['city'],phone_data['state'])
                          )
        map.add_child(marker)
    except KeyError:
        pass
        

map # Finally, with all the coordinates, map will be plotted
